In [ ]:
# !git clone 'https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch.git'

In [ ]:
import sys
sys.path.insert(0, "../input/yetanothereffdet/Yet-Another-EfficientDet-Pytorch-master")

In [ ]:
!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [ ]:
!pip install webcolors

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import argparse
import datetime
import os
import traceback
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import yaml
from tensorboardX import SummaryWriter
from torch import nn
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from tqdm.autonotebook import tqdm
import albumentations as A
from backbone import EfficientDetBackbone
from efficientdet.dataset import Resizer, Normalizer, Augmenter, collater
from efficientdet.loss import FocalLoss
from utils.sync_batchnorm import patch_replication_callback
from utils.utils import replace_w_sync_bn, CustomDataParallel, get_last_weights, init_weights, boolean_string
import cv2
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
def save_checkpoint(model, name):
    torch.save(model.model.state_dict(), os.path.join(saved_path, name))

Classes for data augmentation and transformation

In [ ]:
class ModelWithLoss(nn.Module):
    def __init__(self, model, debug=False):
        super().__init__()
        self.criterion = FocalLoss()
        self.model = model
        self.debug = debug

    def forward(self, imgs, annotations, obj_list=None):
        _, regression, classification, anchors = self.model(imgs)
        if self.debug:
            cls_loss, reg_loss = self.criterion(classification, regression, anchors, annotations,
                                                imgs=imgs, obj_list=obj_list)
        else:
            cls_loss, reg_loss = self.criterion(classification, regression, anchors, annotations)
        return cls_loss, reg_loss


Dataset Class


In [ ]:
class RoadSignDataset(Dataset):
    def __init__(self,train_df,transforms=None,test=False):
        super().__init__()
        self.filenames = train_df['Filename'].values
        self.df = train_df
        self.transforms = transforms
        
    def __getitem__(self,idx):
        filename = self.filenames[idx]
        img_path = os.path.join(DIR,filename)
        
        cv2_img = cv2.imread(img_path)
        cv2_img = cv2.cvtColor(cv2_img,cv2.COLOR_BGR2RGB)
        cv2_img = cv2_img.astype(np.float32)
        cv2_img /=255.0
        
        record = self.df[self.df['Filename']==filename]
        boxes = record[['Upper left corner X','Upper left corner Y','Lower right corner X','Lower right corner Y','Annotation tag']].values

        for i in range(len(boxes)):
            boxes[i,4] = class2idx[boxes[i,4]] 
             
        boxes =  np.array(list(boxes), dtype=np.float)
        
        sample = {'img': cv2_img, 'annot': boxes}
        
        if self.transforms:
            sample = self.transforms(sample)
        return sample
        
    def __len__(self):
        return len(self.filenames)    


In [ ]:
project = 'coco'                        # help='project file that contains parameters'
compound_coef = 1                       # help='coefficients of efficientdet 0,1,2,3,4'
num_workers = 4                         # help='num_workers of dataloader'
batch_size = 4                         # help='The number of images per batch among all devices'
head_only = False                       # 'whether finetunes only the regressor and the classifier, '
lr = 1e-4
optim = 'adamw'                         # help='select optimizer for training, '
num_epochs = 50                 
val_interval = 1                        # help='Number of epoches between valing phases'
save_interval = 1374                      # help='Number of steps between saving'
es_min_delta = 0.0                      # help='Early stopping\'s parameter: minimum change loss to qualify as an improvement'
es_patience = 0
log_path = 'logs/'                      # help='Early stopping\'s parameter: number of epochs with no improvement after which training will be stopped. Set to 0 to disable this technique.'
weights_path = r'./efficientdet-d1.pth'                      # help='the root folder of dataset'
saved_path = './checkpoints/'                   # help='whether to load weights from a checkpoint, set None to initialize, set \'last\' to load last checkpoint'
debug = False                          # help='whether visualize the predicted boxes of training, the output images will be in test/'
SEED = 42
last_step = 0
os.makedirs(log_path, exist_ok=True)
os.makedirs(saved_path, exist_ok=True)

input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
anchors_scales = '[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]'
anchors_ratios = '[(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]'


In [ ]:
import os
DIR = r'.train_data'
csv_path = os.path.join(DIR,'allAnnotations.csv')

df = pd.read_csv(csv_path,delimiter=';')
df.head()

print("Total number of annotations: ",len(df))
print("Total number of Images: ",len(df['Filename'].unique()))

# create unique integer label for each class starting from 1 since 0 is background 
classes = df['Annotation tag'].unique()

class2idx = {}
counter=1
for c in classes:
    class2idx[c] = counter
    counter+=1

idx2class = {v:k for k,v in class2idx.items()}
num_classes = len(idx2class) +1
print("number of classes are: ",num_classes)

In [ ]:
# split the data into train and validation 70:30
train_df, val_df = train_test_split(df,test_size=0.3,random_state=SEED)
training_params = {'batch_size': batch_size,'shuffle': True, 'drop_last': True, 'collate_fn': collater,'num_workers': num_workers}
val_params = {'batch_size': batch_size, 'shuffle': False, 'drop_last': True,'collate_fn': collater,'num_workers': num_workers}

training_set = RoadSignDataset(train_df,transforms=transforms.Compose([Normalizer(mean=mean, std=std),Augmenter(),Resizer(input_sizes[compound_coef])]))        
training_generator = DataLoader(training_set, **training_params)

val_set =  RoadSignDataset(val_df,transforms=transforms.Compose([Normalizer(mean=mean, std=std),Augmenter(),Resizer(input_sizes[compound_coef])]))
val_generator = DataLoader(val_set, **val_params)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = EfficientDetBackbone(num_classes = num_classes, compound_coef=compound_coef,ratios=eval(anchors_ratios), scales=eval(anchors_scales))
try:
    ret = model.load_state_dict(torch.load(weights_path), strict=False)
except RuntimeError as e:
    print(f'[Warning] Ignoring {e}')
    print(
        '[Warning] Don\'t panic if you see this, this might be because you load a pretrained weights with different number of classes. The rest of the weights should be loaded already.')


writer = SummaryWriter(log_path + f'/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}/')

In [ ]:
 # warp the model with loss function, to reduce the memory usage on gpu0 and speedup
model = ModelWithLoss(model, debug=debug)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
epoch = 0
best_loss = 1e5
best_epoch = 0
step = max(0, last_step)
#model.train()

In [ ]:
num_iter_per_epoch = len(training_generator)
try:
    for epoch in range(num_epochs):
        last_epoch = step // num_iter_per_epoch
        if epoch < last_epoch:
            continue

        epoch_loss = []
        progress_bar = tqdm(training_generator)
        for iter, data in enumerate(progress_bar):
            if iter < step - last_epoch * num_iter_per_epoch:
                progress_bar.update()
                continue
            try:
                imgs = data['img']
                annot = data['annot']

                    # if only one gpu, just send it to cuda:0
                    # elif multiple gpus, send it to multiple gpus in CustomDataParallel, not here
                imgs = imgs.to(device)
                annot = annot.to(device)

                optimizer.zero_grad()
                cls_loss, reg_loss = model(imgs, annot, obj_list=classes)
                cls_loss = cls_loss.mean()
                reg_loss = reg_loss.mean()

                loss = cls_loss + reg_loss
                if loss == 0 or not torch.isfinite(loss):
                    continue

                loss.backward()
                # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
                optimizer.step()

                epoch_loss.append(float(loss))

                progress_bar.set_description(
                    'Step: {}. Epoch: {}/{}. Iteration: {}/{}. Cls loss: {:.5f}. Reg loss: {:.5f}. Total loss: {:.5f}'.format(
                        step, epoch, num_epochs, iter + 1, num_iter_per_epoch, cls_loss.item(),
                        reg_loss.item(), loss.item()))
                writer.add_scalars('Loss', {'train': loss}, step)
                writer.add_scalars('Regression_loss', {'train': reg_loss}, step)
                writer.add_scalars('Classfication_loss', {'train': cls_loss}, step)

                # log learning_rate
                current_lr = optimizer.param_groups[0]['lr']
                writer.add_scalar('learning_rate', current_lr, step)

                step += 1

                if step % save_interval == 0 and step > 0:
                    torch.save(model.state_dict(),f'efficientdet-d{compound_coef}_{epoch}_{step}.pth')

            except Exception as e:
                print('[Error]', traceback.format_exc())
                print(e)
                continue
        scheduler.step(np.mean(epoch_loss))

        if epoch % val_interval == 0:
            model.eval()
            loss_regression_ls = []
            loss_classification_ls = []
            for iter, data in enumerate(val_generator):
                with torch.no_grad():
                    imgs = data['img']
                    annot = data['annot']

                    imgs = imgs.to(device)
                    annot = annot.to(device)

                    cls_loss, reg_loss = model(imgs, annot, obj_list=classes)
                    cls_loss = cls_loss.mean()
                    reg_loss = reg_loss.mean()

                    loss = cls_loss + reg_loss
                    if loss == 0 or not torch.isfinite(loss):
                        continue

                    loss_classification_ls.append(cls_loss.item())
                    loss_regression_ls.append(reg_loss.item())

            cls_loss = np.mean(loss_classification_ls)
            reg_loss = np.mean(loss_regression_ls)
            loss = cls_loss + reg_loss

            print(
                'Val. Epoch: {}/{}. Classification loss: {:1.5f}. Regression loss: {:1.5f}. Total loss: {:1.5f}'.format(
                    epoch, num_epochs, cls_loss, reg_loss, loss))
            writer.add_scalars('Loss', {'val': loss}, step)
            writer.add_scalars('Regression_loss', {'val': reg_loss}, step)
            writer.add_scalars('Classfication_loss', {'val': cls_loss}, step)

            if loss + es_min_delta < best_loss:
                best_loss = loss
                best_epoch = epoch

                save_checkpoint(model, f'efficientdet-d{compound_coef}_{epoch}_{step}.pth')

            model.train()

            # Early stopping
            if epoch - best_epoch > es_patience > 0:
                print('[Info] Stop training at epoch {}. The lowest loss achieved is {}'.format(epoch, best_loss))
                break
except KeyboardInterrupt:
    save_checkpoint(model, f'efficientdet-d{compound_coef}_{epoch}_{step}.pth')
    writer.close()
writer.close()

torch.save(model.state_dict(), os.path.join(saved_path, str(epoch)))